# Battery life cycle prediction - deep learning model

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
import pickle
import matplotlib.pyplot as plt
import gc
from pathlib import Path
from IPython.display import display

import src.constants as cst
import src.features.rebuilding_features as rf
import src.models.train_model as tm
#from src.data.data_class import BatteryData
#from src.data.load_data import DataLoader
#from src.data.wrangle_data import DataWrangler

#from rebuilding_features import load_batches_to_dict
from src.visualization.helpers import print_dict_keys
from os.path import join
import src.models.data_pipeline as dp

import tensorflow as tf
import pandas as pd
import seaborn as sns
from matplotlib import style
import itertools
import warnings
import json
import seaborn as sns
import src.constants as cst

import src.models.data_pipeline as dp  # TODO: Have to refactor this code out of this class.
import src.models.split_model as split_model
import src.models.full_cnn_model as full_cnn_model
from src.models.callbacks import CustomCheckpoints

warnings.filterwarnings('ignore')


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
#session = tf.compat.v1.Session(config=config)
#tf.compat.v1.keras.backend.set_session(session)
import os
from pandas import datetime

DATA_DIR = join("../../data/external")


In [4]:
print (physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
def calculate_steps_per_epoch(data_dir, dataset_config):
    temp_dataset = dp.create_dataset(data_dir=data_dir,
                                     window_size=dataset_config["window_size"],
                                     shift=dataset_config["shift"],
                                     stride=dataset_config["stride"],
                                     batch_size=dataset_config["batch_size"],
                                     repeat=False)
    steps_per_epoch = 0
    for batch in temp_dataset:
        steps_per_epoch += 1
    return steps_per_epoch

def get_tboard_dir():
    run_timestr = datetime.now().strftime("%Y%m%d-%H%M%S")
    tboard_dir = os.path.join(cst.TENSORBOARD_DIR, "jobs", run_timestr)
    return tboard_dir

# Deep Learning model

We build a deep learning model by


In [6]:
args = tm.get_args()
#model_trainer = tm.ModelTrainer(None, args)
#model_trainer.train_and_evaluate()

In [ ]:
scaling_factors = dp.load_scaling_factors()
dataset_dir = cst.TEST_SET
window_size = 20
shift = 5
stride = 1
batch_size = 32
hparams = None
save_from = 80
tboard = get_tboard_dir()

print (tboard)

dataset = dp.create_dataset(dataset_dir,
                            window_size=window_size,
                            shift=shift,  # Can vary during validation
                            stride=stride,
                            batch_size=batch_size,  # Can vary during validation
                            cycle_length=1,  # To match original order (so no files get interleaved)
                            num_parallel_calls=1,  # Has to be equal or below cycle_length
                            shuffle=None,  # To match original order
                            repeat=None)

# Config datasets for consistent usage
ds_config = dict(window_size=window_size,
                 shift=shift,
                 stride=stride,
                 batch_size=batch_size)
ds_train_path = cst.TRAIN_SET
ds_val_path = cst.TEST_SET

# create model
if args.model == 'split_model':
    print("Using split model!")
    model = split_model.create_keras_model(window_size=ds_config["window_size"],
                                           loss=args.loss,
                                           hparams_config=hparams)
if args.model == 'full_cnn_model':
    print("Using full cnn model!")
    model = full_cnn_model.create_keras_model(window_size=ds_config["window_size"],
                                              loss=args.loss,
                                              hparams_config=hparams)

# Calculate steps_per_epoch_train, steps_per_epoch_test
# This is needed, since for counting repeat has to be false
steps_per_epoch_train = calculate_steps_per_epoch(ds_train_path, ds_config)

steps_per_epoch_validate = calculate_steps_per_epoch(ds_val_path, ds_config)

# load datasets
dataset_train = dp.create_dataset(data_dir=ds_train_path,
                                  window_size=ds_config["window_size"],
                                  shift=ds_config["shift"],
                                  stride=ds_config["stride"],
                                  batch_size=ds_config["batch_size"])

dataset_validate = dp.create_dataset(data_dir=ds_val_path,
                                     window_size=ds_config["window_size"],
                                     shift=ds_config["shift"],
                                     stride=ds_config["stride"],
                                     batch_size=ds_config["batch_size"])

# if hparams is passed, we're running a HPO-job
if hparams:
    checkpoint_callback = CustomCheckpoints(save_last_only=True,
                                            log_dir=tboard,
                                            dataset_path=ds_val_path,
                                            dataset_config=ds_config,
                                            save_eval_plot=False)
else:
    checkpoint_callback = CustomCheckpoints(save_best_only=True,
                                            start_epoch=save_from,
                                            log_dir=tboard,
                                            dataset_path=ds_val_path,
                                            dataset_config=ds_config,
                                            save_eval_plot=False)
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=tboard,
                                   histogram_freq=0,
                                   write_graph=False,
                                   ),
    checkpoint_callback,
]

model.summary()

# train model
history = model.fit(
    dataset_train,
    epochs=500,
    steps_per_epoch=steps_per_epoch_train,
    validation_data=dataset_validate,
    validation_steps=steps_per_epoch_validate,
    verbose=2,
    callbacks=callbacks)

mae_current = min(history.history["val_mae_current_cycle"])
mae_remaining = min(history.history["val_mae_remaining_cycles"])

#return mae_current, mae_remaining

Graph\jobs\20230226-060056
Using split model!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Qdlin (InputLayer)             [(None, 20, 1000, 1  0           []                               
                                )]                                                                
                                                                                                  
 Tdlin (InputLayer)             [(None, 20, 1000, 1  0           []                               
                                )]                                                                
                                                                                                  
 detail_concat (Concatenate)    (None, 20, 1000, 2)  0           ['Qdlin[0][0]',                  
                                                

In [ ]:
mae_current = min(history.history["val_mae_current_cycle"])
mae_remaining = min(history.history["val_mae_remaining_cycles"])


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10, 10))
line1, = axes.plot(history.epoch, history.history['loss'], label='train')
line2, = axes.plot(history.epoch, history.history['val_loss'], label='validation')
_ = axes.xaxis.set_label_text("Epoch")
_ = axes.yaxis.set_label_text("Loss")
_ = axes.legend(handles=[line1, line2])


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10, 10))
line1, = axes.plot(history.epoch, history.history['mae_remaining_cycles'], label='train')
line2, = axes.plot(history.epoch, history.history['val_mae_remaining_cycles'], label='validation')
_ = axes.xaxis.set_label_text("Epoch")
_ = axes.yaxis.set_label_text("MAE")
_ = axes.legend(handles=[line1, line2])